# Python 2 to 3 issues

Some brief notes on Python 2to3 issues with examples

## `__cmp__` deprecation
`cmp(A, B)` is now deprecated. At a very crude level you can replace it thus:

In [8]:
%%script python2
eg = ['Ted', 'Chris', 'Jan', 'Pete']
eg2 = ['Tom', 'Dick', 'Harry', 'Bob']
eg.sort()
print(cmp(eg, eg2))

-1


In [9]:
eg = ['Ted', 'Chris', 'Jan', 'Pete']
eg2 = ['Tom', 'Dick', 'Harry', 'Bob']
eg.sort()
# print(cmp(eg, eg2)) # This won't work, feel free to uncomment it and give it a go.
print((eg > eg2) - (eg < eg2))

-1


Sort functions are also gone:

In [10]:
%%script python2
def eg_sort(A, B):
    "Sort by last letter"
    return cmp(A[-1], B[-1])

eg = ['Ted', 'Chris', 'Jan', 'Pete']
eg.sort(eg_sort)
print(eg)

['Ted', 'Pete', 'Jan', 'Chris']


To be replaced by keyword sorting

In [12]:
eg = ['Ted', 'Chris', 'Jan', 'Pete']
# Still sort by last letter, isn't this nicer?
eg.sort(key=lambda x: x[-1])
print(eg)

['Ted', 'Pete', 'Jan', 'Chris']


In cases where you need to translate a non-trivial example of the sorting method the following is often the simplest solution

In [13]:
from functools import cmp_to_key

def eg_sort(A, B):
    "Sort by last letter"
    return (A[-1] > B[-1]) - (A[-1] > B[-1])

eg = ['Ted', 'Chris', 'Jan', 'Pete']
eg.sort(key=cmp_to_key(eg_sort))
print(eg)

['Ted', 'Chris', 'Jan', 'Pete']


Additionally, comparison functions will sometimes fall over when transferred to Python 3 because the objects being compared haven't got the appropriate rich comparison methods (`__gt__`, `__lt__` &c.)

## Encoding issues
in Python 3 you have two "string"-like types, string and byte. You can set them, but they won't concatenate.

In [19]:
# You can concat bytes
x = b'my name is Tim'
y = b'Hello, '
print(y+x)

# ...or strings
x = 'my name is Tim'
y = 'Hello, '
print(y+x)

# But not bytes and strings
x = b'my name is Tim' # byte examples
y = 'Hello, '         # string example
print(y+x)

b'Hello, my name is Tim'
Hello, my name is Tim


TypeError: must be str, not bytes

This mainly causes problems when reading or writing as the behaviour has changed a fair bit. You can encode by using:

In [20]:
x = b'my name is Tim'
print(x.decode('UTF-8'))
x = 'my name is Tim'
print(x.encode('UTF-8'))

my name is Tim
b'my name is Tim'


The use of the 'UTF-8' arg is recommended, otherwise you will use system encoding, which is not likely to be consistent across users!

I've written a good deal more about this in another notebook


## `file` keyword gone!
The `file` keyword has gone, so you can't use it for comparisons such as:

In [21]:
%%script python2
import sys
print(isinstance(sys.stdin, file))

True


You need to consider alternative tests

In [22]:
import sys
from io import TextIOBase
print(isinstance(sys.stdin, TextIOBase))

True


## `print_exc(exc)` wrong

This:
```python
    except Exception as exc:
        traceback.print_exc(exc)
```
has never been deprecated for a long time, and ought to be:
```python
    except Exception as exc:
        traceback.print_exc()
```
It seems to have snuck under the testing radar before, but will now definately raise an error along the lines:
`TypeError: '>=' not supported between instances of 'ConfigProcessError' and 'int'`


## `tuple(self)` in custom exception

worked in Python 3 when replaced with `self.args`. I've not had any luck investigating this.

## `Multiprocessing.Pool issue`
Multiprocessing tasks may fail with `TypeError: cannot serialize '???' object`. This requires further study.

## The way `__slots__` works has changed slightly:

In [57]:
%%python2
class Brian():
    __slots__ = ["x"]
    
    x = 42
    
    def __init__(self):
        self.x = "Romanus te Domus"
        print("Hello Brian!")

Brian()

Hello Brian!


In [63]:
# Doesn't work any-more.
class Brian():
    __slots__ = ["x"]
    
    x = "He's not the messiah"
    
    def __init__(self):
        print("Hello Brian!")

Brian()

ValueError: 'x' in __slots__ conflicts with class variable

In [59]:
class Brian():
    __slots__ = ["x"]
    
    
    def __init__(self):
        self.x = "He's a very naughty boy!"
        print("Hello Brian!")

Brian()

Hello Brian!
